# Chapter 1

In [1]:
import nsfg
import thinkstats2
import sys
import numpy as np
import pandas
from collections import defaultdict

In [2]:
# データの読み込み
dct = thinkstats2.ReadStataDct('2002FemPreg.dct')
raw = dct.ReadFixedWidth('2002FemPreg.dat.gz', compression='gzip')
raw

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,poverty_i,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,3410.389399,3869.349602,6448.271112,2,9,1231
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,7226.301740,8567.549110,12999.542264,2,12,1231
5,6,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,4870.926435,5325.196999,8874.440799,1,23,1231
6,6,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,4870.926435,5325.196999,8874.440799,1,23,1231
7,6,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,0,4870.926435,5325.196999,8874.440799,1,23,1231
8,7,1,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,...,0,0,0,0,3409.579565,3787.539000,6911.879921,2,14,1233
9,7,2,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,...,0,0,0,0,3409.579565,3787.539000,6911.879921,2,14,1233


## 新生児の体重に対する Data Cleanning




In [3]:
# Raw データの新生児体重のポンド部分
raw.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
51.0       1
97.0       1
98.0       1
99.0      57
Name: birthwgt_lb, dtype: int64

### raw データの新生児体重の取り扱いの注意点
- ポンドとオンスに分かれているため、それらを合算する必要がある
- 以下のような特別の符号が含まれることがあり、それらは取り除く必要がある。
    - 97 NOT ASCERTAINED
    - 98 REFUSED
    - 99 DON'T KNOW
- 51.0 ポンドのようにありえないデータが含まれている

In [4]:
# Data Cleanning
interim = raw.copy()

# 新生児の体重について特別な符号をnanに置き換え
# 97 NOT ASCERTAINED
# 98 REFUSED
# 99 DON'T KNOW
na_vals = [97,98, 99]
interim.birthwgt_lb.replace(na_vals, np.nan, inplace=True)
interim.birthwgt_oz.replace(na_vals, np.nan, inplace=True)

interim.loc[interim.birthwgt_lb > 20] = np.nan

# 新生児体重についてポンドとオンス部分を合算して一つのデータに
interim['totalwgt_lb'] = interim.birthwgt_lb + interim.birthwgt_oz /16.0


In [5]:
# Cleanning 後のデータの新生児体重のポンド部分
interim.birthwgt_lb.value_counts().sort_index()

0.0        8
1.0       40
2.0       53
3.0       98
4.0      229
5.0      697
6.0     2223
7.0     3049
8.0     1889
9.0      623
10.0     132
11.0      26
12.0      10
13.0       3
14.0       3
15.0       1
Name: birthwgt_lb, dtype: int64